# User Credentials

In [21]:
# Hard coded
credentials={"username" : "firstname.lastname@noaa.gov", "password" : "NOAA_LDAP_PASSWORD"}
# OR
# Project (Home) Folder from currently opened ArcGIS Pro
home_folder = arcpy.mp.ArcGISProject("CURRENT").homeFolder
# Load user_credentials.py to get user credentials
# def main():
#     credentials={"username" : "firstname.lastname@noaa.gov", "password" : "NOAA_LDAP_PASSWORD"}
#     return credentials
# if __name__ == '__main__':
#     main()
# Imports
import os
# Change Directory
os.chdir(home_folder)
import user_credentials
credentials = user_credentials.main()

# InPort API (Beta) #
###### https://docs.google.com/document/d/1txAdY6YChEcdOoEWquF3GzT6yBvpHOiuOJIQvWNZHaE/edit?tab=t.0#heading=h.mij7lb2wckgv ######

## Loader API Specs ###

### Authentication ###

In [12]:
import requests, json
inport_url = "https://test-www.fisheries.noaa.gov/inport"
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))
if inport_response.status_code == 200:
    print("Authentication successful!")
elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT ###--->>> Authentication successful!

Authentication successful!


### Get Session ID ###
#### /inport/api/get-session-id ####
#### (POST) ####

In [37]:
import requests, json, copy
inport_url = "https://test-www.fisheries.noaa.gov/inport"
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))
if inport_response.status_code == 200:
    print("Authentication successful!")
    _url = "https://test-www.fisheries.noaa.gov/inport/api/get-session-id"
    session_id_response = requests.post(_url, json = credentials)
    session_id_dict = copy.deepcopy(session_id_response.json())
    print(session_id_dict)
    session_id_response.close()
    del _url, session_id_response
elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT ###--->>> Authentication successful!
# {'sessionId': 'b7f9fb6bf34a6be8b357020e'}

Authentication successful!
{'sessionId': '22469789aebb6c1fcaad6669'}


### Endpoints ###

### Load InPort XML ###
#### /inport/api/load-inport-xml ####
#### (POST) ####

In [13]:
# Imports
import requests, json, copy

# InPort URL
inport_url = "https://test-www.fisheries.noaa.gov/inport"

# Use requests get method to log into InPort
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))

# Test if successful i.e. status code 200
if inport_response.status_code == 200:
    print("Authentication successful!")

    # InPort URL to get a session id
    _url = "https://test-www.fisheries.noaa.gov/inport/api/get-session-id"
    
    # Use requests post method to get a session id for credentials pasted to InPort
    session_id_response = requests.post(_url, json = credentials)
    
    # Make a deep copy of the response, using the method json() to return a Python dictionary
    session_id_dict = copy.deepcopy(session_id_response.json())
    
    # Close the response object and then delete variable
    session_id_response.close()
    del _url, session_id_response

    xml = rf"{home_folder}\Metadata Export\PolygonExample_InPort.xml"

    # Opening a file for reading
    with open(xml, "r") as file:
        content = file.read()
    # Insert XML content from file into the dictionary, using 'xml' as the key,     
    session_id_dict["xml"] = content
 
    _url = "https://test-www.fisheries.noaa.gov/inport/api/load-inport-xml"
    # headers = {"Content-Type": "application/json"}     
    # response = requests.post(url, json=payload, headers=headers)
    # response = requests.post(_url, json=session_id_dict, headers=headers)
    # Use request post method to load content to InPort
    response = requests.post(_url, json=session_id_dict)
    
    if response.status_code == 200:
        print(response)
        _dict = copy.deepcopy(response.json())
        response.close()
    
        for key in _dict:
            print(key)
            if isinstance(_dict[key], dict):
                for k in _dict[key]:
                    print(f"\t{k}: {_dict[key][k]}")
            elif isinstance(_dict[key], list):
                for i in range(0, len(_dict[key])):
                    for k in _dict[key][i]:
                        print(f"\t{k}: {_dict[key][i][k]}")
                    print()
  
    elif response.status_code == 400:
        print(session_id_dict)    
        print(response.content)        
    
    else:
        print(f"Something is wrong with load-inport-xml!!\n\tStatus Code: {response.status_code}\n\tContent: {response.content}")

    del response, _url

elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT ###--->>> Authentication successful!

Authentication successful!
<Response [200]>
catId
errors
	summary: Catalog Item ID
	details: The uploaded file does not specify a catalog item ID to update, or a parent catalog item to create a child under. The catalog item to be processed must be specified.
	sourceField: /inport-metadata/item-identification/catalog-item-id
	destinationField: Section: Item Identification | Field: ID

warnings
childItems


### Get path from root ###
#### /inport/api/get-path-from-root ####
#### (POST) ####

In [14]:
import requests, json, copy
inport_url = "https://test-www.fisheries.noaa.gov/inport"
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))
if inport_response.status_code == 200:
    print("Authentication successful!")
    _url = "https://test-www.fisheries.noaa.gov/inport/api/get-session-id"
    session_id_response = requests.post(_url, json = credentials)
    session_id_dict = copy.deepcopy(session_id_response.json())
    print(session_id_dict)
    session_id_response.close()
    del _url, session_id_response    
    session_id_dict["catId"] = "75728"
    _url = "https://test-www.fisheries.noaa.gov/inport/api/get-path-from-root"
    response = requests.get(_url, params = session_id_dict)        
    #print(response.json())
    _dict = copy.deepcopy(response.json())
    
    response.close()
    del response, _url
    
    for key in _dict:
        print(key)
        if isinstance(_dict[key], dict):
            for k in _dict[key]:
                print(f"\t{k}: {_dict[key][k]}")
        elif isinstance(_dict[key], list):
            for i in range(0, len(_dict[key])):
                for k in _dict[key][i]:
                    print(f"\t{k}: {_dict[key][i][k]}")
                print()


elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT 
# Authentication successful!
# {'sessionId': '75c3628752d91a126fcbded22998f3e'}
# path
# 	catId: 1196
# 	catItemType: LIB
# 	title: Office of Science and Technology Metadata Library
# 	shortName: OST Metadata Library
# 	ownerOrgAcronym: OST

Authentication successful!
{'sessionId': 'aa8d4c2f9108a0fb2ad5930b'}
path
	catId: 1196
	catItemType: LIB
	title: Office of Science and Technology Metadata Library
	shortName: OST Metadata Library
	ownerOrgAcronym: OST

	catId: 10443
	catItemType: PRJ
	title: ST6: Science Information Division
	shortName: None
	ownerOrgAcronym: OST

	catId: 75791
	catItemType: PRJ
	title: NMFS HQ GIS
	shortName: None
	ownerOrgAcronym: OST

	catId: 65591
	catItemType: PRJ
	title: NOAA Distribution Mapping and Analysis Portal (DisMAP)
	shortName: DisMAP
	ownerOrgAcronym: OST



## Contacts API Specs ###

### Endpoints ###

#### /inport/api/find-contact ####
#### (POST) ####

In [15]:
import requests, json, copy
inport_url = "https://test-www.fisheries.noaa.gov/inport"
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))
if inport_response.status_code == 200:
    print("Authentication successful!")
    _url = "https://test-www.fisheries.noaa.gov/inport/api/get-session-id"
    session_id_response = requests.post(_url, json = credentials)
    session_id_dict = copy.deepcopy(session_id_response.json())
    print(session_id_dict)
    session_id_response.close()
    del _url, session_id_response    
    
    session_id_dict["contactType"]  = "Person"
    session_id_dict["emailAddress"] = "john.f.kennedy@noaa.gov"
    
    _url = "https://test-www.fisheries.noaa.gov/inport/api/find-contact"
    response = requests.get(_url, params = session_id_dict)        
    #print(response.json())
    _dict = copy.deepcopy(response.json())
    
    response.close()
    del response, _url
    
    for key in _dict:
        print(key)
        if isinstance(_dict[key], dict):
            for k in _dict[key]:
                print(f"\t{k}: {_dict[key][k]}")
        elif isinstance(_dict[key], list):
            for i in range(0, len(_dict[key])):
                for k in _dict[key][i]:
                    print(f"\t{k}: {_dict[key][i][k]}")


elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT 
# Authentication successful!
# {'sessionId': '75c3628752d91a126fcbded22998f3e'}
# results
# 	lastName: Kennedy
# 	firstName: John
# 	middleName: F
# 	phone: 301-427-8149
# 	emailAddress: john.f.kennedy@noaa.gov
# 	address: 1315 East-West Highway
# 	city: Silver Spring
# 	stateProvince: MD
# 	postalCode: 20910
# 	country: USA
# 	contactType: Person
# 	rorMaxLength: 9
# 	orcIdMaxLength: 19

Authentication successful!
{'sessionId': '9c7613eff8bd9b5a37663422'}
results
	lastName: Kennedy
	firstName: John
	middleName: F
	phone: 301-427-8149
	emailAddress: john.f.kennedy@noaa.gov
	address: 1315 East-West Highway
	city: Silver Spring
	stateProvince: MD
	postalCode: 20910
	country: USA
	contactType: Person
	orcIdMaxLength: 19
	rorMaxLength: 9


### Endpoints ###

#### /inport/api/find-person ####
#### (POST) ####

In [16]:
import requests, json, copy
inport_url = "https://test-www.fisheries.noaa.gov/inport"
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))
if inport_response.status_code == 200:
    print("Authentication successful!")
    _url = "https://test-www.fisheries.noaa.gov/inport/api/get-session-id"
    session_id_response = requests.post(_url, json = credentials)
    session_id_dict = copy.deepcopy(session_id_response.json())
    #print(session_id_dict)
    session_id_response.close()
    del _url, session_id_response       
    
    session_id_dict["firstName"]  = "John"
    session_id_dict["middleName"] = "F"
    session_id_dict["lastName"]   = "Kennedy"
    print(session_id_dict)

    _url = "https://test-www.fisheries.noaa.gov/inport/api/find-person"
    response = requests.get(_url, params = session_id_dict)        
    #print(response.json())
    _dict = copy.deepcopy(response.json())
    response.close()
    del response, _url
    
    for key in _dict:
        print(key)
        if isinstance(_dict[key], dict):
            for k in _dict[key]:
                print(f"\t{k}: {_dict[key][k]}")
        elif isinstance(_dict[key], list):
            for i in range(0, len(_dict[key])):
                for k in _dict[key][i]:
                    print(f"\t{k}: {_dict[key][i][k]}")


elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT 
# Authentication successful!
# {'sessionId': '75c3628752d91a126fcbded22998f3e'}
# results
# 	lastName: Kennedy
# 	firstName: John
# 	middleName: F
# 	phone: 301-427-8149
# 	emailAddress: john.f.kennedy@noaa.gov
# 	address: 1315 East-West Highway
# 	city: Silver Spring
# 	stateProvince: MD
# 	postalCode: 20910
# 	country: USA
# 	contactType: Person
# 	rorMaxLength: 9
# 	orcIdMaxLength: 19

Authentication successful!
{'sessionId': '4d89909c9d0db2ed5bd3d318', 'firstName': 'John', 'middleName': 'F', 'lastName': 'Kennedy'}
{'results': [{'lastName': 'Kennedy', 'firstName': 'John', 'middleName': 'F', 'phone': '301-427-8149', 'emailAddress': 'john.f.kennedy@noaa.gov', 'address': '1315 East-West Highway', 'city': 'Silver Spring', 'stateProvince': 'MD', 'postalCode': '20910', 'country': 'USA', 'contactType': 'Person', 'orcIdMaxLength': 19, 'rorMaxLength': 9}]}
results
	lastName: Kennedy
	firstName: John
	middleName: F
	phone: 301-427-8149
	emailAddress: john.f.kennedy@noaa.gov
	address: 1315 East-West Highway
	city: Silver Spring
	stateProvince: MD
	postalCode: 20910
	country: USA
	contactType: Person
	orcIdMaxLength: 19
	rorMaxLength: 9


#### /inport/api/find-organization ####
#### (POST) ####

In [17]:
import requests, json, copy
inport_url = "https://test-www.fisheries.noaa.gov/inport"
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))
if inport_response.status_code == 200:
    print("Authentication successful!")
    _url = "https://test-www.fisheries.noaa.gov/inport/api/get-session-id"
    session_id_response = requests.post(_url, json = credentials)
    session_id_dict = copy.deepcopy(session_id_response.json())
    #print(session_id_dict)
    session_id_response.close()
    del _url, session_id_response       
    
    session_id_dict["organizationName"]  = "NMFS Office of Science and Technology"
    print(session_id_dict)

    _url = "https://test-www.fisheries.noaa.gov/inport/api/find-organization"
    response = requests.get(_url, params = session_id_dict)        
    #print(response.json())
    _dict = copy.deepcopy(response.json())
    response.close()
    del response, _url
    
    for key in _dict:
        print(key)
        if isinstance(_dict[key], dict):
            for k in _dict[key]:
                print(f"\t{k}: {_dict[key][k]}")
        elif isinstance(_dict[key], list):
            for i in range(0, len(_dict[key])):
                for k in _dict[key][i]:
                    print(f"\t{k}: {_dict[key][i][k]}")


elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT 
# Authentication successful!
# {'sessionId': 'f0a9190579b46115943bad0f', 'organizationName': 'NMFS Office of Science and Technology'}
# {'results': [{'organizationName': 'NMFS Office of Science and Technology', 'noaaAcronym': 'OST', 'phone': '301-427-8100', 'address': '1315 East West Highway', 'city': 'Silver Spring', 'stateProvince': 'MD', 'postalCode': '20910', 'country': 'USA', 'url': 'https://www.st.nmfs.noaa.gov/', 'businessHours': '0700 - 1800 EST/EDT', 'contactType': 'Organization', 'rorMaxLength': 9, 'orcIdMaxLength': 19}]}
# results
# 	organizationName: NMFS Office of Science and Technology
# 	noaaAcronym: OST
# 	phone: 301-427-8100
# 	address: 1315 East West Highway
# 	city: Silver Spring
# 	stateProvince: MD
# 	postalCode: 20910
# 	country: USA
# 	url: https://www.st.nmfs.noaa.gov/
# 	businessHours: 0700 - 1800 EST/EDT
# 	contactType: Organization
# 	rorMaxLength: 9
# 	orcIdMaxLength: 19

Authentication successful!
{'sessionId': 'a7c1690764d8523dbacf1bec', 'organizationName': 'NMFS Office of Science and Technology'}
{'results': [{'organizationName': 'NMFS Office of Science and Technology', 'noaaAcronym': 'OST', 'phone': '301-427-8100', 'address': '1315 East West Highway', 'city': 'Silver Spring', 'stateProvince': 'MD', 'postalCode': '20910', 'country': 'USA', 'url': 'https://www.st.nmfs.noaa.gov/', 'businessHours': '0700 - 1800 EST/EDT', 'contactType': 'Organization', 'orcIdMaxLength': 19, 'rorMaxLength': 9}]}
results
	organizationName: NMFS Office of Science and Technology
	noaaAcronym: OST
	phone: 301-427-8100
	address: 1315 East West Highway
	city: Silver Spring
	stateProvince: MD
	postalCode: 20910
	country: USA
	url: https://www.st.nmfs.noaa.gov/
	businessHours: 0700 - 1800 EST/EDT
	contactType: Organization
	orcIdMaxLength: 19
	rorMaxLength: 9


#### /inport/api/find-organization-acronym ####
#### (POST) ####

In [18]:
import requests, json, copy
inport_url = "https://test-www.fisheries.noaa.gov/inport"
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))
if inport_response.status_code == 200:
    print("Authentication successful!")
    _url = "https://test-www.fisheries.noaa.gov/inport/api/get-session-id"
    session_id_response = requests.post(_url, json = credentials)
    session_id_dict = copy.deepcopy(session_id_response.json())
    #print(session_id_dict)
    session_id_response.close()
    del _url, session_id_response       
    
    session_id_dict["noaaAcronym"] = "OST"
    print(session_id_dict)

    _url = "https://test-www.fisheries.noaa.gov/inport/api/find-organization-acronym"
    response = requests.get(_url, params = session_id_dict)        
    print(response.json())
    _dict = copy.deepcopy(response.json())
    response.close()
    del response, _url
    
    for key in _dict:
        print(key)
        if isinstance(_dict[key], dict):
            for k in _dict[key]:
                print(f"\t{k}: {_dict[key][k]}")
        elif isinstance(_dict[key], list):
            for i in range(0, len(_dict[key])):
                for k in _dict[key][i]:
                    print(f"\t{k}: {_dict[key][i][k]}")


elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT 
# Authentication successful!
# {'sessionId': 'f0a9190579b46115943bad0f', 'organizationName': 'NMFS Office of Science and Technology'}
# {'results': [{'organizationName': 'NMFS Office of Science and Technology', 'noaaAcronym': 'OST', 'phone': '301-427-8100', 'address': '1315 East West Highway', 'city': 'Silver Spring', 'stateProvince': 'MD', 'postalCode': '20910', 'country': 'USA', 'url': 'https://www.st.nmfs.noaa.gov/', 'businessHours': '0700 - 1800 EST/EDT', 'contactType': 'Organization', 'rorMaxLength': 9, 'orcIdMaxLength': 19}]}
# results
# 	organizationName: NMFS Office of Science and Technology
# 	noaaAcronym: OST
# 	phone: 301-427-8100
# 	address: 1315 East West Highway
# 	city: Silver Spring
# 	stateProvince: MD
# 	postalCode: 20910
# 	country: USA
# 	url: https://www.st.nmfs.noaa.gov/
# 	businessHours: 0700 - 1800 EST/EDT
# 	contactType: Organization
# 	rorMaxLength: 9
# 	orcIdMaxLength: 19

Authentication successful!
{'sessionId': '2e20148df04a3a6f63360d93', 'noaaAcronym': 'OST'}
{'results': [{'organizationName': 'NMFS Office of Science and Technology', 'noaaAcronym': 'OST', 'phone': '301-427-8100', 'address': '1315 East West Highway', 'city': 'Silver Spring', 'stateProvince': 'MD', 'postalCode': '20910', 'country': 'USA', 'url': 'https://www.st.nmfs.noaa.gov/', 'businessHours': '0700 - 1800 EST/EDT', 'contactType': 'Organization', 'orcIdMaxLength': 19, 'rorMaxLength': 9}]}
results
	organizationName: NMFS Office of Science and Technology
	noaaAcronym: OST
	phone: 301-427-8100
	address: 1315 East West Highway
	city: Silver Spring
	stateProvince: MD
	postalCode: 20910
	country: USA
	url: https://www.st.nmfs.noaa.gov/
	businessHours: 0700 - 1800 EST/EDT
	contactType: Organization
	orcIdMaxLength: 19
	rorMaxLength: 9


#### /inport/api/find-position
#### (POST)
#### NOTE: not sure how to look for "position"

In [19]:
import requests, json, copy
inport_url = "https://test-www.fisheries.noaa.gov/inport"
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))
if inport_response.status_code == 200:
    print("Authentication successful!")
    _url = "https://test-www.fisheries.noaa.gov/inport/api/get-session-id"
    session_id_response = requests.post(_url, json = credentials)
    session_id_dict = copy.deepcopy(session_id_response.json())
    #print(session_id_dict)
    session_id_response.close()
    del _url, session_id_response       
    
    session_id_dict["positionName"] = "Biologist"
    print(session_id_dict)

    _url = "https://test-www.fisheries.noaa.gov/inport/api/find-position"
    response = requests.get(_url, params = session_id_dict)        
    print(response.json())
    _dict = copy.deepcopy(response.json())
    response.close()
    del response, _url
    
    for key in _dict:
        print(key)
        if isinstance(_dict[key], dict):
            for k in _dict[key]:
                print(f"\t{k}: {_dict[key][k]}")
        elif isinstance(_dict[key], list):
            for i in range(0, len(_dict[key])):
                for k in _dict[key][i]:
                    print(f"\t{k}: {_dict[key][i][k]}")

elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT 
# Authentication successful!
# {'sessionId': '78cda61c957924113e94fdd6', 'positionName': 'Biologist'}
# {'results': []}
# results

Authentication successful!
{'sessionId': '78cda61c957924113e94fdd6', 'positionName': 'Biologist'}
{'results': []}
results


#### /inport/api/add-contact ####
#### (POST) ####
##### Note: not used at this time #####

## Search API Specs ###

### Endpoints ###

#### /inport/api/search ####
#### (GET) ####

In [20]:
import requests, json, copy
inport_url = "https://test-www.fisheries.noaa.gov/inport"
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))
if inport_response.status_code == 200:
    print("Authentication successful!")
    _url = "https://test-www.fisheries.noaa.gov/inport/api/get-session-id"
    session_id_response = requests.post(_url, json = credentials)
    session_id_dict = copy.deepcopy(session_id_response.json())
    #print(session_id_dict)
    session_id_response.close()
    del _url, session_id_response       
    
    session_id_dict["keywords"]    = "environment"
    session_id_dict["org"]         = "OST"
    session_id_dict["catItemType"] = "DS,ENT,PRJ"
    print(session_id_dict)

    _url = "https://test-www.fisheries.noaa.gov/inport/api/search"
    response = requests.get(_url, params = session_id_dict)        
    #print(response.json())
    _dict = copy.deepcopy(response.json())
    response.close()
    del response, _url
    
    for key in _dict:
        print(key)
        if isinstance(_dict[key], dict):
            for k in _dict[key]:
                print(f"\t{k}: {_dict[key][k]}")
        elif isinstance(_dict[key], list):
            for i in range(0, len(_dict[key])):
                for k in _dict[key][i]:
                    print(f"\t{k}: {_dict[key][i][k]}")

elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT 
# Authentication successful!
# {'sessionId': '13a880c7ba85d337a572d92f', 'keywords': 'environment', 'org': 'OST', 
# 'catItemType': 'DS,ENT,PRJ'}
# results
# 	catId: 20784
# 	catItemType: Project
# 	org: OST
# 	title: Large Marine Ecosystems (LME)
# 	abstractValue: Large Marine Ecosystems (LME) are ocean regions of 200,000 km2 or greater 
#   that are defined by ecological criteria, including bathymetry, hydrography, marine 
#   productivity, and trophically linked populations.  Since 1984, NOAA?s LME Program has 
#   developed ecosystem management tools, initiated projects that have been funded by partner 
#   organizations, and provided training for developing country part...
	

Authentication successful!
{'sessionId': '13a880c7ba85d337a572d92f', 'keywords': 'environment', 'org': 'OST', 'catItemType': 'DS,ENT,PRJ'}
results
	catId: 20784
	catItemType: Project
	org: OST
	title: Large Marine Ecosystems (LME)
	abstractValue: Large Marine Ecosystems (LME) are ocean regions of 200,000 km2 or greater that are defined by ecological criteria, including bathymetry, hydrography, marine productivity, and trophically linked populations.  Since 1984, NOAA?s LME Program has developed ecosystem management tools, initiated projects that have been funded by partner organizations, and provided training for developing country part...
	catId: 21376
	catItemType: Project
	org: OST
	title: Process Products (PPS) System
	abstractValue: This legacy system was originally developed using Oracle Forms 3.2 and over the years has been migrated many times up to it's current 11g version. In it's original release, the system functioned in a Client Server environment but now serves its interna